## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Barrow,US,71.2906,-156.7887,-27.38,68,0,8.05,clear sky
1,1,Paros,GR,37.0833,25.1500,54.30,66,61,19.57,broken clouds
2,2,Kaitangata,NZ,-46.2817,169.8464,59.94,72,99,7.29,overcast clouds
3,3,Aklavik,CA,68.2191,-135.0107,-24.77,97,100,11.74,overcast clouds
4,4,Atuona,PF,-9.8000,-139.0333,77.32,80,20,13.96,light rain


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Atuona,PF,-9.8000,-139.0333,77.32,80,20,13.96,light rain
18,18,Hithadhoo,MV,-0.6000,73.0833,82.22,73,64,12.75,broken clouds
22,22,Trincomalee,LK,8.5711,81.2335,80.80,68,73,13.29,broken clouds
23,23,Mahebourg,MU,-20.4081,57.7000,82.26,94,75,9.22,broken clouds
27,27,Port Blair,IN,11.6667,92.7500,78.84,69,20,11.50,few clouds
29,29,Touros,BR,-5.1989,-35.4608,81.50,78,6,10.60,clear sky
32,32,Faanui,PF,-16.4833,-151.7500,80.51,76,100,12.44,light rain
34,34,Saint-Philippe,RE,-21.3585,55.7679,80.15,80,31,11.05,scattered clouds
41,41,Bathsheba,BB,13.2167,-59.5167,77.61,73,40,16.11,scattered clouds
44,44,Meiktila,MM,20.8667,95.8667,75.97,27,10,7.61,clear sky


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                168
City                   168
Country                168
Lat                    168
Lng                    168
Max Temp               168
Humidity               168
Cloudiness             168
Wind Speed             168
Current Description    168
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#THERE ARE NO EMPTY ROWS

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Atuona,PF,77.32,light rain,-9.8000,-139.0333,
18,Hithadhoo,MV,82.22,broken clouds,-0.6000,73.0833,
22,Trincomalee,LK,80.80,broken clouds,8.5711,81.2335,
23,Mahebourg,MU,82.26,broken clouds,-20.4081,57.7000,
27,Port Blair,IN,78.84,few clouds,11.6667,92.7500,
29,Touros,BR,81.50,clear sky,-5.1989,-35.4608,
32,Faanui,PF,80.51,light rain,-16.4833,-151.7500,
34,Saint-Philippe,RE,80.15,scattered clouds,-21.3585,55.7679,
41,Bathsheba,BB,77.61,scattered clouds,13.2167,-59.5167,
44,Meiktila,MM,75.97,clear sky,20.8667,95.8667,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
    
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   168
Country                168
Max Temp               168
Current Description    168
Lat                    168
Lng                    168
Hotel Name             168
dtype: int64

In [18]:
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Atuona,PF,77.32,light rain,-9.8000,-139.0333,Villa Enata
18,Hithadhoo,MV,82.22,broken clouds,-0.6000,73.0833,Scoop Guest House
22,Trincomalee,LK,80.80,broken clouds,8.5711,81.2335,Welcombe Hotel
23,Mahebourg,MU,82.26,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
27,Port Blair,IN,78.84,few clouds,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"


In [19]:
clean_hotel_df.count()

City                   162
Country                162
Max Temp               162
Current Description    162
Lat                    162
Lng                    162
Hotel Name             162
dtype: int64

In [20]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))